# <center>Data Mining project: Discover and describe areas of interest<br> and events from geo-located parsed_data</center>

## #0: Import Dataset and Libraries

## Setting Environment

In [1]:
! python -m venv dataMiningEnv
#Activate windows
#! dataMiningEnv\Scripts\activate.bat

#Activate mac
! source dataMiningEnv/bin/activate


## Importing Libraries

- ### Installs

In [1]:
# installation of required libraries and dependencies
# numeric calculations
! pip install numpy==1.26.0 
# data frames 
! pip install pandas==2.1.1 
# machine learning algorithms 
! pip install scikit-learn==1.5.1 
! pip install scipy==1.12.0
# plotting 
! pip install plotly==5.24.1 
! pip install matplotlib==3.8.0 
! pip install seaborn==0.13.2 
! pip install plotly-express==0.4.1 
! pip install chart-studio==1.1.0 
# web app library 
! pip install streamlit==1.37.1 
# association rules
! pip install mlxtend==0.23.3
# Language processing
! pip install nltk
! python -m nltk.downloader popular # popular functions


You should consider upgrading via the '/Users/saadelghissassi/Desktop/Data-Lyon-Mining_4IF-master/dataMiningEnv/bin/python -m pip install --upgrade pip' command.
You should consider upgrading via the '/Users/saadelghissassi/Desktop/Data-Lyon-Mining_4IF-master/dataMiningEnv/bin/python -m pip install --upgrade pip' command.
You should consider upgrading via the '/Users/saadelghissassi/Desktop/Data-Lyon-Mining_4IF-master/dataMiningEnv/bin/python -m pip install --upgrade pip' command.
You should consider upgrading via the '/Users/saadelghissassi/Desktop/Data-Lyon-Mining_4IF-master/dataMiningEnv/bin/python -m pip install --upgrade pip' command.
You should consider upgrading via the '/Users/saadelghissassi/Desktop/Data-Lyon-Mining_4IF-master/dataMiningEnv/bin/python -m pip install --upgrade pip' command.
You should consider upgrading via the '/Users/saadelghissassi/Desktop/Data-Lyon-Mining_4IF-master/dataMiningEnv/bin/python -m pip install --upgrade pip' command.
You should consider upgradin

- ### Imports

In [2]:
import pandas as pd
import re
from sklearn.preprocessing import StandardScaler
import sklearn.cluster as cl
import nltk


## Importing Data

In [3]:
# Charger les données avec low_memory=False pour éviter les avertissements
data = pd.read_csv("data/flickr_data2.csv", sep=",", low_memory=False)

## #1: Discovering areas of interests using clustering

## Understanding the Data

--Write something to describe this part of the report--

## Data clearing and preparation

In [4]:
# Étape 1 : Nettoyage initial des colonnes
# Supprimer les espaces supplémentaires dans les noms des colonnes
parsed_data = data
parsed_data.columns = parsed_data.columns.str.strip()

# Convertir les colonnes temporelles en numériques
# Liste des colonnes temporelles
time_columns = [
    'date_taken_minute', 'date_taken_hour', 'date_taken_day',
    'date_taken_month', 'date_taken_year',
    'date_upload_minute', 'date_upload_hour', 'date_upload_day',
    'date_upload_month', 'date_upload_year'
]

# Convertir chaque colonne en int64, remplacer les erreurs par 0
for col in time_columns:
    parsed_data[col] = pd.to_numeric(data[col], errors='coerce').fillna(0).astype('int64')

In [5]:
# Étape 2 : Suppression des doublons basés sur l'identifiant unique
# Sauvegarder les doublons pour audit futur
parsed_data[parsed_data['id'].duplicated(keep='first')].sort_values("id").to_csv(
    "data/parsed_lines/duplicatedId.csv", index=False
)

# Supprimer les doublons
parsed_data = parsed_data[~parsed_data['id'].duplicated(keep='first')]

In [6]:
# Étape 3 : Gestion des colonnes inutilisées ou corrompues
# Identifier les colonnes inutiles
unused_columns = ["Unnamed: 16", "Unnamed: 17", "Unnamed: 18", 
                  "date_upload_minute", "date_upload_hour", "date_upload_day", 
                  "date_upload_month", "date_upload_year"]

# Sauvegarder les données corrompues
parsed_data[parsed_data[unused_columns].notnull().any(axis=1)].to_csv(
    "data/parsed_lines/corrupted_data.csv"
, index=False)

# Supprimer les colonnes inutilisées et les lignes corrompues
parsed_data = parsed_data[]
parsed_data = parsed_data.drop(columns=unused_columns)

SyntaxError: invalid syntax (4018163999.py, line 13)

In [7]:
# Étape 4 : Nettoyage des coordonnées GPS
# Définir les limites géographiques de Lyon
lyon_lat_min, lyon_lat_max = 45.69, 45.85
lyon_lon_min, lyon_lon_max = 4.78, 4.92

# Filtrer les données pour garder uniquement les points dans Lyon
parsed_data = parsed_data[
    (parsed_data['lat'] >= lyon_lat_min) & 
    (parsed_data['lat'] <= lyon_lat_max) &
    (parsed_data['long'] >= lyon_lon_min) &
    (parsed_data['long'] <= lyon_lon_max)
]

In [10]:
# Étape 5 : Gestion des valeurs manquantes
# Supprimer les lignes avec des valeurs manquantes dans latitude/longitude
parsed_data = parsed_data.dropna(subset=['lat', 'long'])

# Remplir les valeurs manquantes pour les colonnes textuelles par des chaînes vides
parsed_data['tags'] = parsed_data['tags'].fillna('')
parsed_data['title'] = parsed_data['title'].fillna('')

In [21]:
# Étape 6 : Suppression des doublons géographiques et temporels
# Identifier les doublons basés sur latitude, longitude, et date
# --> PASSER ÇA EN POST-PROCESSING
print(parsed_data.columns)


Index(['id', 'user', 'lat', 'long', 'tags', 'title', 'date_taken_minute',
       'date_taken_hour', 'date_taken_day', 'date_taken_month',
       'date_taken_year', 'date_upload_minute', 'date_upload_hour',
       'date_upload_day', 'date_upload_month', 'date_upload_year',
       'Unnamed: 16', 'Unnamed: 17', 'Unnamed: 18'],
      dtype='object')


In [24]:
# Étape 7 : Filtrage des dates incohérentes
# Garder uniquement les dates raisonnables (entre 2009 et 2025)
parsed_data = parsed_data[
    (parsed_data['date_taken_year'] >= 2009) & (parsed_data['date_taken_year'] <= 2025) &
    (parsed_data['date_taken_month'] >= 1) & (parsed_data['date_taken_month'] <= 12) &
    (parsed_data['date_taken_day'] >= 1) & (parsed_data['date_taken_day'] <= 31) &
    (parsed_data['date_taken_hour'] >= 0) & (parsed_data['date_taken_hour'] <= 23) &
    (parsed_data['date_taken_minute'] >= 0) & (parsed_data['date_taken_minute'] <= 59) 
]


In [27]:
# Étape 8 : Nettoyage des colonnes textuelles
# Fonction pour nettoyer le texte
def clean_text(text):
    # Supprimer les caractères spéciaux et passer en minuscules
    return re.sub(r'[^a-zA-Z0-9, ]', '', text).lower()

# Appliquer le nettoyage sur les colonnes textuelles
parsed_data['tags'] = parsed_data['tags'].apply(clean_text)
parsed_data['title'] = parsed_data['title'].apply(clean_text)

In [29]:
# Étape 9 : Normalisation des coordonnées GPS
# Normaliser latitude et longitude pour les algorithmes de clustering
scaler = StandardScaler()
parsed_data[['lat', 'long']] = scaler.fit_transform(parsed_data[['lat', 'long']])

In [ ]:
# Résumé final des données nettoyées
print("Données nettoyées :")
print(parsed_data.info())

Données nettoyées :
<class 'pandas.core.frame.DataFrame'>
Index: 388524 entries, 0 to 420239
Data columns (total 19 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   id                  388524 non-null  int64  
 1   user                388524 non-null  object 
 2   lat                 388524 non-null  float64
 3   long                388524 non-null  float64
 4   tags                388524 non-null  object 
 5   title               388524 non-null  object 
 6   date_taken_minute   388524 non-null  int64  
 7   date_taken_hour     388524 non-null  int64  
 8   date_taken_day      388524 non-null  int64  
 9   date_taken_month    388524 non-null  int64  
 10  date_taken_year     388524 non-null  int64  
 11  date_upload_minute  388524 non-null  int64  
 12  date_upload_hour    388524 non-null  int64  
 13  date_upload_day     388524 non-null  int64  
 14  date_upload_month   388524 non-null  int64  
 15  date_upload_year   

In [31]:
# Sauvegarder les données nettoyées pour les prochaines étapes
parsed_data.to_csv("data/cleaned_flickr_data.csv", index=False)